In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
from scipy.stats import zscore
from IPython.display import display

In [16]:
# df = pd.read_csv(Path('../data/2023basketballsheet.csv'), index_col=1)[lambda df_: df_["R#"] < 120]
df = (
    pd.read_csv(Path('../data/2023hashtagbasketballprojections.csv'), index_col=2)
    [lambda df_: df_["R#"] < 120]
    .sort_values("ADP")
)

In [17]:
df.head()

,R#,ADP,POS,TEAM,GP,MPG,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO,TOTAL
PLAYER,,,,,,,,,,,,,,,,
Nikola Jokic,1,1.1,C,DEN,72,33.6,0.624(9.2/14.8),0.821(5.0/6.1),1.0,24.5,11.7,9.7,1.4,0.7,3.7,14.56
Joel Embiid,2,2.6,C,PHI,67,34.2,0.537(10.8/20.0),0.836(9.9/11.9),1.2,32.6,10.2,4.2,1.1,1.6,3.3,13.62
Luka Doncic,6,3.2,PG,DAL,66,35.8,0.492(11.0/22.4),0.761(7.8/10.2),3.0,32.7,8.8,8.1,1.3,0.5,3.6,9.78
Jayson Tatum,5,4.5,"SF,PF",BOS,75,36.5,0.466(9.8/21.0),0.858(6.6/7.7),3.3,29.5,8.2,4.8,1.1,0.7,3.0,9.82
Tyrese Haliburton,4,5.7,"PG,SG",IND,73,34.7,0.488(9.0/18.4),0.857(2.6/3.0),2.8,23.3,4.0,10.5,1.7,0.4,2.6,9.92


In [18]:
STAT_COLS = ["FG%", "FT%", "5pm", "PTS", "TREB", "AST", "STL", "BLK", "TO"]
POSITIONS = ["PG", "SG", "SF", "PF", "C"]

Clean data

In [19]:
def format_percentages(val):
    if "(" in val:
        return float(val[0:val.index('(')])
    else:
        return float(val)
    
def encode_positions(val):
    positions = {
        pos: False
        for pos in POSITIONS
    }
    for code in val.split(","):
        positions[code] = True
    return pd.Series(positions)

def clean_df(df):
    df["FG%"] = df["FG%"].apply(format_percentages)
    df["FT%"] = df["FT%"].apply(format_percentages)   
    #df["TO"] = -1 * df["TO"]
    positions = df["POS"].apply(encode_positions)
    df = df.merge(positions, left_index=True, right_index=True)
    return df

In [20]:
df = df.pipe(clean_df)

Compute zscores amongst players for each positions

In [36]:
positions_zscores = {}
for pos in POSITIONS:
    positions_zscores[pos] = (
        df.loc[df[pos], STAT_COLS]
        .apply(zscore)
        .assign(ADP=df.loc[df[pos], "ADP"])
    )

For each player, collect the zscores. 
For players with multiple positions, we take the most favorable zscores

In [37]:
summary_pos_zscores = {}
for player in df.index:
    to_compare = []
    valid_positions = []
    for pos in POSITIONS:
        if player in positions_zscores[pos].index:
            to_compare.append(positions_zscores[pos].loc[[player]])
            valid_positions.append(pos)
    summary_zscore = pd.concat(to_compare).max()
    summary_zscore["POS"] = ",".join(valid_positions)
    summary_pos_zscores[player] = summary_zscore

Join and summarize zscore data against fantasy rank, team, games played.

Re-rank players by the total z-score, compute rank deltas (how much old rank and new rank differ)

In [38]:
summary_pos_zscores = (
    df[["R#", "TEAM", "GP"]].merge(
        (
            pd.DataFrame.from_dict(summary_pos_zscores, orient="index")
            .assign(total_zscore=lambda df_: df_[STAT_COLS].sum(axis=1))
        ),
        left_index=True, right_index=True
    )
    .sort_values("total_zscore", ascending=False)
    .rename(columns={
        "R#": "old_rank",
    })
    .assign(
        zbased_rank=lambda df_: df_["total_zscore"].rank(ascending=False),
        rank_delta=lambda df_: df_["old_rank"] - df_["zbased_rank"]
    )
)

In [39]:
summary_pos_zscores.head(20)

,old_rank,TEAM,GP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO,ADP,POS,total_zscore,zbased_rank,rank_delta
Nikola Jokic,1,DEN,72,0.655268,1.098496,0.416501,1.448477,1.560428,3.522957,2.674205,-0.960202,2.210788,1.1,C,12.626919,1.0,0.0
Jayson Tatum,5,BOS,75,-0.397222,1.079366,2.047789,2.029944,1.860541,0.794501,0.705095,0.821066,1.250785,4.5,"SF,PF",10.191866,2.0,3.0
Luka Doncic,6,DAL,66,1.344199,-1.579522,0.770377,2.229197,3.487210,1.141870,0.480479,0.592516,1.516990,3.2,PG,9.983315,3.0,3.0
Kevin Durant,10,PHO,60,1.995349,1.759890,0.405949,1.230077,0.638954,0.933492,-0.742206,2.291039,1.380056,10.8,"SF,PF",9.892601,4.0,6.0
Giannis Antetokounmpo,37,MIL,65,1.074311,-0.460928,0.271299,2.467776,2.170254,1.496890,0.455432,0.179613,2.087062,10.1,"PF,C",9.741709,5.0,32.0
LaMelo Ball,13,CHA,69,-1.020066,0.503364,1.600674,0.690067,2.040901,1.982992,1.510076,0.108829,1.944744,12.6,"PG,SG",9.361582,6.0,7.0
Joel Embiid,2,PHI,67,-0.411883,1.267999,0.706905,2.871980,0.603007,0.737115,1.342941,0.363948,1.715884,2.6,C,9.197897,7.0,-5.0
Domantas Sabonis,17,SAC,72,2.099855,0.217082,-0.309510,0.569771,2.577911,2.307317,0.455432,-0.634912,1.344706,21.4,"PF,C",8.627654,8.0,9.0
Shai Gilgeous-Alexander,3,OKC,67,1.115399,1.008306,-1.415089,1.960044,0.433892,0.400818,1.166877,2.527260,0.938842,6.8,"PG,SG",8.136349,9.0,-6.0
Anthony Edwards,21,MIN,76,-0.120281,-1.038745,1.081906,1.304572,0.809437,0.863996,1.875942,1.662730,1.638599,14.6,"SG,SF",8.078156,10.0,11.0


In [40]:
(
    summary_pos_zscores[summary_pos_zscores["old_rank"]<30]
    .sort_values("rank_delta", ascending=False)
    .head(20)
)

,old_rank,TEAM,GP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO,ADP,POS,total_zscore,zbased_rank,rank_delta
Paul George,26,LAC,60,-0.920745,1.225193,1.461418,0.792054,1.060642,0.933492,2.152397,-0.268182,1.638599,30.3,"SG,SF,PF",8.074866,11.0,15.0
LeBron James,29,LAL,56,1.098135,-0.492320,0.875047,1.496699,1.429393,1.628445,-0.380380,-0.158916,1.380056,25.5,"SF,PF",6.876158,17.0,12.0
Anthony Edwards,21,MIN,76,-0.120281,-1.038745,1.081906,1.304572,0.809437,0.863996,1.875942,1.662730,1.638599,14.6,"SG,SF",8.078156,10.0,11.0
Domantas Sabonis,17,SAC,72,2.099855,0.217082,-0.309510,0.569771,2.577911,2.307317,0.455432,-0.634912,1.344706,21.4,"PF,C",8.627654,8.0,9.0
LaMelo Ball,13,CHA,69,-1.020066,0.503364,1.600674,0.690067,2.040901,1.982992,1.510076,0.108829,1.944744,12.6,"PG,SG",9.361582,6.0,7.0
Kevin Durant,10,PHO,60,1.995349,1.759890,0.405949,1.230077,0.638954,0.933492,-0.742206,2.291039,1.380056,10.8,"SF,PF",9.892601,4.0,6.0
Karl-Anthony Towns,23,MIN,70,0.160645,1.234098,2.013725,0.956402,0.590581,0.939722,-0.432077,-0.309102,1.097254,27.8,"PF,C",6.251249,18.0,5.0
Jayson Tatum,5,BOS,75,-0.397222,1.079366,2.047789,2.029944,1.860541,0.794501,0.705095,0.821066,1.250785,4.5,"SF,PF",10.191866,2.0,3.0
Luka Doncic,6,DAL,66,1.344199,-1.579522,0.770377,2.229197,3.487210,1.141870,0.480479,0.592516,1.516990,3.2,PG,9.983315,3.0,3.0
Lauri Markkanen,25,UTA,66,0.585441,1.273801,1.578692,0.963454,1.716825,-0.984113,-0.742206,0.331075,-0.171197,25.4,"SF,PF",4.551774,23.0,2.0


In [42]:
for pos, subdf in positions_zscores.items():
    subdf.index.rename(pos, inplace=True)
    display(subdf.head(20)[["ADP", *STAT_COLS]])
    print("--")

,ADP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
PG,,,,,,,,,,
Luka Doncic,3.2,1.344199,-1.579522,0.770377,2.229197,3.487210,1.141870,0.480479,0.592516,1.516990
Tyrese Haliburton,5.7,1.191666,0.440246,0.551830,0.569110,-0.369612,2.515549,1.853275,0.108829,0.214853
Stephen Curry,6.6,0.734066,1.639484,2.628023,1.399153,0.915995,-0.289045,-0.892318,-0.374857,1.126349
Shai Gilgeous-Alexander,6.8,1.115399,1.008306,-1.415089,1.681721,0.433892,-0.289045,1.166877,2.527260,0.605494
Damian Lillard,9.6,-0.028600,1.471170,1.644563,1.611079,-0.369612,0.569504,-1.235517,-0.374857,0.865921
LaMelo Ball,12.6,-1.020066,0.503364,1.535290,0.586770,2.040901,1.428054,1.510076,0.108829,1.516990
Kyrie Irving,15.2,1.115399,1.513249,0.988923,1.169567,0.273192,-0.346282,0.137280,1.559888,-0.306002
Donovan Mitchell,19.7,0.924733,0.503364,1.535290,1.363832,-0.048210,-0.975884,1.166877,-0.374857,0.345066
Fred VanVleet,21.0,-2.164065,1.092463,0.879650,-0.296255,-0.449963,0.168848,1.510076,0.592516,-0.436216


--


,ADP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
SG,,,,,,,,,,
Tyrese Haliburton,5.7,0.794534,0.434131,0.590903,0.669583,-0.697791,2.985036,1.647995,-0.268182,0.507741
Shai Gilgeous-Alexander,6.8,0.718300,1.006844,-1.428639,1.960044,0.139558,0.400818,1.049932,2.145458,0.938842
LaMelo Ball,12.6,-1.416270,0.497765,1.600674,0.690067,1.814256,1.982992,1.348964,-0.268182,1.944744
Anthony Edwards,14.6,-0.120281,-1.432489,0.927493,1.304572,0.809437,0.031643,1.647995,1.662730,1.513643
Kyrie Irving,15.2,0.718300,1.515922,1.039690,1.366022,-0.027912,0.348078,0.152838,1.180002,-0.067060
Devin Booker,17.0,0.756417,0.518977,0.029919,1.468440,0.055823,0.769992,-0.146193,-0.268182,0.507741
Mikal Bridges,19.1,0.299009,0.921997,0.142116,1.099737,-0.195381,-0.759444,-0.146193,0.697274,-0.641861
Donovan Mitchell,19.7,0.527713,0.497765,1.600674,1.591341,-0.362851,-0.232052,1.049932,-0.750910,0.651441
Desmond Bane,20.4,0.108423,1.134113,1.264084,0.526198,0.474498,-0.443009,0.152838,-0.268182,-0.067060


--


,ADP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
SF,,,,,,,,,,
Jayson Tatum,4.5,-0.397222,0.635551,1.308708,1.882144,1.860541,0.794501,0.065250,0.821066,1.250785
Kevin Durant,10.8,1.995349,1.425669,-0.278905,1.095894,0.638954,0.933492,-1.141878,2.291039,1.380056
Anthony Edwards,14.6,-0.482671,-1.038745,1.081906,1.301816,0.135948,0.863996,1.875942,1.311057,1.638599
Devin Booker,17.0,0.499992,0.691988,0.174699,1.451578,-0.510775,1.836932,0.065250,-0.648907,0.733701
Mikal Bridges,19.1,-0.012702,1.049423,0.288100,1.114614,-0.726349,-0.178434,0.065250,0.331075,-0.300468
Desmond Bane,20.4,-0.226324,1.237546,1.422109,0.590447,-0.151485,0.238538,0.367032,-0.648907,0.216616
Lauri Markkanen,25.4,0.585441,0.861299,0.855104,0.833810,1.716825,-1.220864,-1.141878,0.331075,-0.171197
LeBron James,25.5,1.098135,-1.189244,0.174699,1.357977,1.429393,1.628445,-0.840096,-0.158916,1.380056
Kawhi Leonard,26.6,1.226308,0.729613,-0.278905,0.702768,0.423380,0.030052,0.668814,-0.158916,-0.429739


--


,ADP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
PF,,,,,,,,,,
Jayson Tatum,4.5,-0.547913,1.079366,2.047789,2.029944,0.437709,0.702299,0.705095,-0.309102,1.021273
Giannis Antetokounmpo,10.1,1.074311,-1.756151,-0.766793,2.182300,2.170254,1.225668,-0.018555,0.179613,1.741085
Kevin Durant,10.8,0.496277,1.759890,0.405949,1.230077,-0.428563,0.818603,-0.742206,0.179613,1.141241
Anthony Davis,11.6,1.018372,-0.524726,-1.470439,1.134854,2.068339,-0.460744,0.705095,1.971567,-0.058446
Jaren Jackson Jr.,15.0,-0.305512,0.058580,0.171401,0.144543,-0.122820,-1.391177,0.343270,3.437712,-0.178415
Domantas Sabonis,21.4,2.099855,-0.783974,-1.235890,0.125498,2.577911,2.156101,-0.018555,-0.634912,1.021273
Lauri Markkanen,25.4,-0.119049,1.273801,1.578692,0.963454,0.335795,-0.984113,-0.742206,-0.472007,-0.298384
LeBron James,25.5,0.104706,-0.492320,0.875047,1.496699,0.131966,1.400124,-0.380380,-0.634912,1.141241
Jimmy Butler,26.8,0.291168,1.095569,-1.235890,0.468298,-0.836221,0.818603,2.514222,-0.797816,-0.418353


--


,ADP,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
C,,,,,,,,,,
Nikola Jokic,1.1,0.655268,1.098496,0.416501,1.448477,1.560428,3.522957,2.674205,-0.960202,2.210788
Joel Embiid,2.6,-0.411883,1.267999,0.706905,2.871980,0.603007,0.737115,1.342941,0.363948,1.715884
Giannis Antetokounmpo,10.1,-0.215625,-0.460928,0.271299,2.467776,1.496600,1.496890,0.455432,-0.518819,2.087062
Anthony Davis,11.6,-0.252424,0.397885,-0.599914,1.501199,1.368944,0.027992,1.342941,1.099586,0.231172
Jaren Jackson Jr.,15.0,-1.123317,0.804692,1.432916,0.587345,-1.375663,-0.782435,0.899187,2.423736,0.107446
Domantas Sabonis,21.4,0.459011,0.217082,-0.309510,0.569771,2.007224,2.307317,0.455432,-1.254458,1.344706
Karl-Anthony Towns,27.8,-0.816664,1.234098,2.013725,0.956402,-0.482070,0.939722,-0.432077,-0.960202,1.097254
Victor Wembanyama,30.1,-1.429970,0.623889,0.416501,0.183141,-0.673554,-0.478525,-0.432077,1.393842,-0.140006
Pascal Siakam,30.6,-1.135583,0.443086,0.706905,1.026698,-0.928866,1.294284,0.899187,-1.254458,0.602350


--


In [43]:
positions_zscores["C"].sort_values("FT%", ascending=False)

,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO,ADP
C,,,,,,,,,,
Kristaps Porzingis,-1.135583,1.460102,1.578119,0.481901,-0.801210,-0.225267,0.011678,0.216820,-0.140006,41.4
Joel Embiid,-0.411883,1.267999,0.706905,2.871980,0.603007,0.737115,1.342941,0.363948,1.715884,2.6
Karl-Anthony Towns,-0.816664,1.234098,2.013725,0.956402,-0.482070,0.939722,-0.432077,-0.960202,1.097254,27.8
Nikola Jokic,0.655268,1.098496,0.416501,1.448477,1.560428,3.522957,2.674205,-0.960202,2.210788,1.1
Jonas Valanciunas,-0.301488,1.087196,-0.164308,-0.379231,0.411523,-0.377222,-1.763340,-0.960202,0.107446,102.4
Nikola Vucevic,-0.988390,0.928994,1.142512,0.200715,1.113632,0.230598,-0.432077,-0.960202,-0.140006,35.7
Jaren Jackson Jr.,-1.123317,0.804692,1.432916,0.587345,-1.375663,-0.782435,0.899187,2.423736,0.107446,15.0
Brook Lopez,-0.657205,0.793392,1.287714,-0.344083,-2.141599,-0.833087,-1.319586,1.246714,-0.882362,57.5
Bam Adebayo,-0.289222,0.657789,-1.035521,0.692790,0.347695,0.331902,2.230450,-0.813074,0.849802,35.6
